# Задание 5.

Загрузите [данные по изменению температуры поверхности земли](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data). Для этого может понадобится зарегистрироваться на [Kaggle](https://kaggle.com). Затем нужно будет работать с данными, которые содержатся в файле **GlobalLandTemperaturesByMajorCity.csv**

**NB** Все подсчеты необходимо делать с помощью `PySpark`, без применения `pandas api`. Можно использоать `SQL`.

In [5]:
import zipfile
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession

%matplotlib inline

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("PySpark") \
    .getOrCreate()

with zipfile.ZipFile("archive.zip") as z:
    with z.open("GlobalLandTemperaturesByMajorCity.csv") as f:
        pandas_df = pd.read_csv(f)
        df = spark.createDataFrame(pandas_df)

df.show(10)

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1849-01-01|            26.704|                        1.435|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-02-01|            27.434|                        1.362|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-03-01|            28.101|                        1.612|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-04-01|             26.14|           1.3869999999999998|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-05-01|            25.427|                          1.2|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-06-01|            24.844|                        1.402|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-07-01|24.058000000000003|                        1.254|Abidjan|Côte

# Задание 4.1 (1 балл)

В последующих заданиях будут учитываться данные начиная с 01.01.1950. Для этого создайте новый `DataFrame`, в котором удалены все строки до 01.01.1950. Используйте созданный DataFrame в последующих заданиях.  

In [6]:
from pyspark.sql.functions import col

# 转换日期列为日期类型
# Преобразуйте столбец с датой в тип дата
df = df.withColumn("dt", col("dt").cast("date"))

# 过滤1950年1月1日之后的数据
# Отфильтруйте данные после 1 января 1950 года
df_filtered = df.filter(col("dt") >= "1950-01-01")

df_filtered.show(10)


+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1950-01-01|26.773000000000003|                        0.239|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-02-01|            27.527|                        0.348|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-03-01|            28.344|                        0.431|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-04-01|             27.83|                        0.467|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-05-01|            26.896|                        0.248|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-06-01|            25.454|                        0.209|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-07-01|            24.878|                        0.403|Abidjan|Côte

# Задание 4.2 (2 балла)

Найдите город, для которого выборочная дисперсия температур на приведенных данных максимальна. 

In [21]:
from pyspark.sql.functions import col, var_samp

# 转换AverageTemperature列为double类型并过滤掉包含空值的行
# Преобразуйте столбец AverageTemperature в тип double и удалите строки, содержащие пустые значения
df_filtered_non_null = df_filtered.filter(df_filtered["AverageTemperature"].isNotNull())
df_filtered_non_null = df_filtered_non_null.withColumn("AverageTemperature", col("AverageTemperature").cast("double"))

# 检查是否存在非数值数据，如NaN和无穷大
# Проверьте наличие нечисловых данных, таких как NaN и бесконечность
df_filtered_non_null = df_filtered_non_null.filter(~col("AverageTemperature").isin([float("nan"), float("inf"), float("-inf")]))

# 计算每个城市的温度样本方差
# Вычислите выборочную дисперсию температуры для каждого города
city_variance = df_filtered_non_null.groupBy("City").agg(var_samp("AverageTemperature").alias("TemperatureVariance"))

# 找出温度方差最大的城市
# Найдите город с максимальной дисперсией температуры
max_variance_city = city_variance.orderBy(col("TemperatureVariance").desc()).first()

print("City with maximum temperature variance:", max_variance_city)


City with maximum temperature variance: Row(City='Harbin', TemperatureVariance=218.898615951821)


# Задание 4.3 (2 баллов)

Посчитайте данные по среднегодовой температуре в Санкт-Петербурге. Определите года, в которых средняя температура была выше, чем в предыдущем  и последующем году. 

In [23]:
from pyspark.sql.functions import year, avg, lag, lead
from pyspark.sql.window import Window

# 过滤圣彼得堡的数据
# Отфильтруйте данные по Санкт-Петербургу
spb_data = df_filtered_non_null.filter(col("City") == "Saint Petersburg")

# 计算每年的平均温度
# Вычислите среднегодовую температуру
spb_yearly_avg = spb_data.groupBy(year("dt").alias("Year")).agg(avg("AverageTemperature").alias("YearlyAvgTemp"))

# 增加前一年和后一年的温度用于比较
# Добавьте температуру предыдущего и следующего года для сравнения
windowSpec = Window.orderBy("Year")
spb_yearly_avg = spb_yearly_avg.withColumn("PrevYearTemp", lag("YearlyAvgTemp").over(windowSpec))
spb_yearly_avg = spb_yearly_avg.withColumn("NextYearTemp", lead("YearlyAvgTemp").over(windowSpec))

# 过滤平均温度高于前一年和后一年的年份
# Отфильтруйте годы с температурой выше, чем в предыдущем и следующем году
spb_special_years = spb_yearly_avg.filter(
    (col("YearlyAvgTemp") > col("PrevYearTemp")) & 
    (col("YearlyAvgTemp") > col("NextYearTemp"))
)

spb_special_years.show()


+----+------------------+------------------+------------------+
|Year|     YearlyAvgTemp|      PrevYearTemp|      NextYearTemp|
+----+------------------+------------------+------------------+
|1953| 4.840083333333333| 3.749333333333333|             4.454|
|1957| 4.856249999999999|2.5654999999999997|3.2618333333333336|
|1959| 5.026000000000001|3.2618333333333336| 4.108249999999999|
|1961| 5.842083333333334| 4.108249999999999| 4.038250000000001|
|1964| 4.414166666666666|3.4835833333333333| 3.702166666666667|
|1967| 4.618666666666667|2.8411666666666666|3.4489999999999994|
|1972| 5.294333333333333| 4.157750000000001| 4.421500000000001|
|1975| 5.885249999999999| 5.642583333333334|2.5989166666666668|
|1977|3.9462500000000005|2.5989166666666668|2.6957500000000003|
|1979|4.0605833333333345|2.6957500000000003|3.6558333333333333|
|1983| 5.237583333333333| 4.482166666666667| 5.008583333333333|
|1986| 4.054749999999999|2.6400833333333336|2.2548333333333335|
|1989| 6.585916666666667| 4.930166666666

# Задание 4.4 (4 балла)

Найдите города, для которых: 
1. Разница между максимальным и минимальным значением среднегодовой температуры в выборке максимальна.
2. Самая большая средняя разница между средней температурой января и средней температурой июля.
3. Наибольшее среднее количество месяцев с отрицательной температурой в году.

In [24]:
from pyspark.sql.functions import max, min, avg, year, month, sum, col

## 1. 最大和最小年平均温度值之间的差异最大的城市
## 1. Города с наибольшей разницей между максимальными и минимальными значениями среднегодовой температуры
# 计算每个城市每年的平均温度
# Рассчитайте среднегодовую температуру для каждого города
city_yearly_avg = df_filtered_non_null.groupBy("City", year("dt").alias("Year")).agg(avg("AverageTemperature").alias("YearlyAvgTemp"))

# 计算每个城市的最大和最小年平均温度
# Рассчитайте максимальную и минимальную среднегодовую температуру для каждого города
city_temp_range = city_yearly_avg.groupBy("City").agg(
    max("YearlyAvgTemp").alias("MaxYearlyAvgTemp"),
    min("YearlyAvgTemp").alias("MinYearlyAvgTemp")
)

# 计算每个城市的年平均温度范围（最大值 - 最小值）
# Рассчитайте разницу между максимальным и минимальным годовым средними температурами для каждого города
city_temp_range = city_temp_range.withColumn("TempRange", col("MaxYearlyAvgTemp") - col("MinYearlyAvgTemp"))

# 找出年平均温度范围最大的城市
# Найдите город с максимальной разницей между максимальным и минимальным годовым средними температурами
max_temp_range_city = city_temp_range.orderBy(col("TempRange").desc()).first()
print("City with maximum difference between max and min yearly average temperatures:", max_temp_range_city)


## 2. 一月平均温度和七月平均温度之间的平均差异最大的城市
## 2. Города с наибольшей средней разницей между средней температурой января и средней температурой июля
# 添加月份列
# Добавьте столбцы месяцев
df_filtered_non_null = df_filtered_non_null.withColumn("Month", month("dt"))

# 计算每个城市一月和七月的平均温度
# Рассчитайте среднюю температуру для каждого города в январе и июле
january_avg = df_filtered_non_null.filter(col("Month") == 1).groupBy("City").agg(avg("AverageTemperature").alias("JanuaryAvgTemp"))
july_avg = df_filtered_non_null.filter(col("Month") == 7).groupBy("City").agg(avg("AverageTemperature").alias("JulyAvgTemp"))

# 计算一月和七月平均温度差
# Рассчитайте разницу между средней температурой в январе и июле
jan_july_diff = january_avg.join(july_avg, "City").withColumn("TempDiff", col("JulyAvgTemp") - col("JanuaryAvgTemp"))

# 找出一月和七月平均温度差最大的城市
# Найдите город с максимальной разницей между средней температурой в январе и июле
max_temp_diff_city = jan_july_diff.orderBy(col("TempDiff").desc()).first()
print("City with maximum average temperature difference between January and July:", max_temp_diff_city)


## 3. 每年平均月数中温度为负数的最多的城市
## 3. Города с наибольшим количеством отрицательных температур в среднем за несколько месяцев в году
# 添加一个列标识温度是否为负数
# Добавьте столбец, определяющий, является ли температура отрицательной или нет
df_filtered_non_null = df_filtered_non_null.withColumn("NegativeTemp", (col("AverageTemperature") < 0).cast("int"))

# 计算每个城市每年温度为负数的月份数
# Рассчитайте количество месяцев с отрицательной температурой для каждого города
city_neg_temp_count = df_filtered_non_null.groupBy("City", year("dt").alias("Year")).agg(sum("NegativeTemp").alias("NegTempMonths"))

# 计算每个城市的平均负温月份数
# Рассчитайте среднее количество месяцев с отрицательной температурой для каждого города
avg_neg_temp_count = city_neg_temp_count.groupBy("City").agg(avg("NegTempMonths").alias("AvgNegTempMonths"))

# 找出平均负温月份数最多的城市
# Найдите город с наибольшим средним количеством месяцев с отрицательной температурой
max_neg_temp_city = avg_neg_temp_count.orderBy(col("AvgNegTempMonths").desc()).first()
print("City with the highest average number of months with negative temperatures:", max_neg_temp_city)


City with maximum difference between max and min yearly average temperatures: Row(City='Mashhad', MaxYearlyAvgTemp=15.938500000000001, MinYearlyAvgTemp=10.6885, TempRange=5.250000000000002)
City with maximum average temperature difference between January and July: Row(City='Harbin', JanuaryAvgTemp=-18.578421875000004, JulyAvgTemp=23.414296874999998, TempDiff=41.99271875)
City with the highest average number of months with negative temperatures: Row(City='Harbin', AvgNegTempMonths=4.90625)
